In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class loss_function:
    def __init__(self, df, column, business_days, npath, rfr, risk_neutral, strike, option_type, obs_price):
        self.df = df
        self.column = column
        self.business_days = business_days
        self.npath = npath
        self.rfr = rfr
        self.risk_neutral = risk_neutral
        self.strike = strike
        self.option_type = option_type
        self.obs_price = obs_price
        self.maturity = business_days / 252

    def heston_pso_minimizer(self, bounds, iterr):

        def objective_function(params):
            # Ensure params is 5D
            if params.ndim == 1:
                params = params.reshape(1, -1)

            loss_values = []  # Store loss for each particle
            for particle in params:
                rho, kappa, sigma_sigma, theta, inst_vol = particle
                loss_option_price = 0
                for i in range(len(self.business_days)):

                    ''' -----------------------------------------------'''

                    model_option_price = heston_options_value(self.df[self.column].iloc[-1], self.strike[i], self.maturity[i], self.rfr, rho, kappa, sigma_sigma, self.rfr, theta, inst_vol, self.option_type[i]).heston_p1p2()

                    ''' -----------------------------------------------'''
                    # Simulate future prices using MC and calculate option payoff

                    ''' -----------------------------------------------'''

                    loss_option_price += (model_option_price - self.obs_price[i])**2

                loss_values.append(loss_option_price)  # Add loss for this particle

            if len(loss_values) == 1:
                return loss_values[0]
            else:
                return np.array(loss_values)  # Return array of losses for all particles

        # -----------------------------
        # 2. PSO Setup
        # -----------------------------
        dimensions = len(bounds[0])
        n_particles = 20

        # PSO parameters
        options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


        optimizer = ps.single.GlobalBestPSO(
            n_particles=n_particles,
            dimensions=dimensions,
            options=options,
            bounds=bounds
        )

        pos_history = []
        cost_history = []
        mean_history = []  # Store mean of each parameter

        optimizer.reset()

        for _ in range(iterr):
            cost, pos = optimizer.optimize(objective_function, iters=1, verbose=False)
            swarm_pos = optimizer.swarm.position.copy()
            pos_history.append(swarm_pos)
            cost_history.append(cost)
            mean_history.append(np.mean(swarm_pos, axis=0))  # Mean over particles

        # Convert histories to arrays
        mean_history = np.array(mean_history)     # shape: (iterations, dimensions)
        cost_history = np.array(cost_history)

        # Plot Mean of Particles Across Iterations
        fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

        for d in range(dimensions):
            axs[0].plot(mean_history[:, d], label=f"Param {d+1}")
        axs[0].set_ylabel("Mean Parameter Value")
        axs[0].set_title("Mean of Each Parameter over Iterations")
        axs[0].legend()
        axs[0].grid(True)

        axs[1].plot(cost_history, color='purple')
        axs[1].set_xlabel("Iteration")
        axs[1].set_ylabel("Best Cost")
        axs[1].set_title("Cost Convergence over Iterations")
        axs[1].grid(True)

        plt.tight_layout()
        plt.show()

        return pos  # Best parameters found